In [ ]:
# Load required libraries
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
# Load 2 data sets
app_rec = pd.read_csv(r'C:\Users\shaha\OneDrive\Documents\credit_score\application_record.csv')

cre_rec = pd.read_csv(r'C:\Users\shaha\OneDrive\Documents\credit_score\credit_record.csv')

In [ ]:
# Check the size of each data set

In [ ]:
print(app_rec.shape)
print(cre_rec.shape)

# A - app_rec Data Cleaning

In [ ]:
app_rec.columns

In [ ]:
app_rec.head()

In [ ]:
# Create code for 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE'
# I may use a function to create numeric value for each category or use one hot encoding technique to process the data

In [ ]:
app_rec.info()

In [ ]:
#Check the Numeric fields
app_rec.describe().T

In [ ]:
# Change the data type of the ID

In [ ]:
# Create moth_employed from DAYS_EMPLOYED.

In [ ]:
# Create an employement flag fom DAYS_EMPLOYED.

In [ ]:
# Create Age bucket based on DAYS_BIRTH.

In [ ]:
# Create income bucket based on AMT_INCOME_TOTAL

In [ ]:
# Create Count of Children bucket based on CNT_CHILDREN

In [ ]:
app_rec.CNT_CHILDREN.value_counts()

In [ ]:
#check data type in each column and verify it

In [ ]:
#check Null Values

In [ ]:
#Check Dublicate rows

In [ ]:
app_rec.ID.duplicated().sum()

In [ ]:
#check missing values

In [ ]:
for name in app_rec.columns:
    print(app_rec[name].value_counts())

In [ ]:
# Check Outliers

In [ ]:
# Remove Outlyers

# B - cre_rec Data Cleaning & Transforming

In [ ]:
# check the column name of both datasets

In [ ]:
cre_rec.columns

In [ ]:
#Transpose the credit record 

In [ ]:
cre_rec.sort_values(["ID","MONTHS_BALANCE"],ascending=False,    inplace=False).head(10).set_index('ID')

In [ ]:
cre_rec.pivot(index="ID", columns='MONTHS_BALANCE', values= 'STATUS')

In [ ]:
cre_rec.groupby(by=["ID"])

In [ ]:
cre_rec.ID.duplicated().count()

In [ ]:
cre_rec.shape

In [ ]:
cre_rec.STATUS.value_counts()

# C - Merge the Data Sets

In [ ]:
data = pd.merge(app_rec,cre_rec)

In [ ]:
data

In [ ]:
# Merge 2 datasets based on the id

In [ ]:
#Save the Cleaned Data

# D - Model Development

In [ ]:
# Reload the Cleaned Data

In [ ]:
#Reload the Cleaned Data

In [ ]:
#Load Required librarys
import sksklearn 

In [ ]:
#train model 1

In [ ]:
#Test model 1

In [ ]:
#evaluate model 1

In [1]:
pwd

'C:\\Users\\shaha\\Documents'

In [ ]:
# test